In [5]:
import pickle
import pandas as pd

pd.options.display.float_format = '{:.3f}'.format

In [6]:
with open('Downloads/dz_8.09_result.pkl', 'rb') as f:
    dz_809 = pickle.load(f)

In [7]:
res = pd.DataFrame(dz_809)

In [8]:
res.T

,duration,accuracy_train,accuracy_test,f1_train,f1_test
dtc_min_leaf=6,0.0045 sec,0.910,0.837,0.917,0.853
rfc_max_feat=6_max_sampl=0.6,0.0641 sec,0.973,0.891,0.975,0.906
rfc_default,0.1274 sec,1.000,0.897,1.000,0.910
baggcl_max_feat=6_max_sampl=0.6,0.0619 sec,0.896,0.886,0.908,0.902
stack_svc,2.2455 sec,0.981,0.880,0.983,0.895
stack_baggcl,0.9115 sec,0.936,0.880,0.942,0.896
gbc_max_feat=10_min_leaf=5,0.0226 sec,0.891,0.891,0.903,0.905
